In [1]:
from unsloth import FastLanguageModel 
import torch

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.3.1+cu121)
    Python  3.9.18 (you have 3.9.2)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/mnt/bn/data-tns-live-llm/leon/datasets/repetition/cherry-alpaca-3",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.347 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [3]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
inputs = tokenizer("generate some instruction tuning sample used for your training", return_tensors="pt")
input_ids = inputs.input_ids.cuda()
generate_ids = model.generate(input_ids, max_length=2048, repetition_penalty=1.1, streamer = text_streamer)
outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(outputs)

In [18]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = "LEON24/files"
folder_path = "/mnt/bn/data-tns-live-llm/leon/datasets/files"
api.create_repo(repo_id, exist_ok=True, private=True)
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_id,
)

code_cherry_1000.pt:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

code_MoDS_1000.pt:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

MathInstruct.json:   0%|          | 0.00/212M [00:00<?, ?B/s]

cherry_1000.pt:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

MoDS_1000.pt:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

code_data_cherry.pt:   0%|          | 0.00/25.8M [00:00<?, ?B/s]

math_data_cherry.pt:   0%|          | 0.00/38.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LEON24/files/commit/6463c78cb88699b514be76499756d33d9c6bcc64', commit_message='Upload folder using huggingface_hub', commit_description='', oid='6463c78cb88699b514be76499756d33d9c6bcc64', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, T5ForConditionalGeneration
import torch
model = T5ForConditionalGeneration.from_pretrained("google/t5-xl-lm-adapt", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/t5-xl-lm-adapt")

/home/tiger/.local/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
args = """--model_name_or_path google/t5-xl-lm-adapt --data_path /mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet --per_device_train_batch_size 256 --eval_strategy no --num_train_epochs 3 --learning_rate 2e-5 --warmup_ratio 0.03 --output_dir /mnt/bn/data-tns-live-llm/leon datasets/p3_exp1 --logging_steps 100 --save_strategy "steps" --save_steps 100 --save_total_limit 3 --model_max_length 1024""".split(" ")
print(args)

['--model_name_or_path', 'google/t5-xl-lm-adapt', '--data_path', '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet', '--per_device_train_batch_size', '256', '--eval_strategy', 'no', '--num_train_epochs', '3', '--learning_rate', '2e-5', '--warmup_ratio', '0.03', '--output_dir', '/mnt/bn/data-tns-live-llm/leon', 'datasets/p3_exp1', '--logging_steps', '100', '--save_strategy', '"steps"', '--save_steps', '100', '--save_total_limit', '3', '--model_max_length', '1024']


In [10]:
print("['--model_name_or_path', 'google/t5-xl-lm-adapt', '--data_path', '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet', '--per_device_train_batch_size', '256', '--eval_strategy', 'no', '--num_train_epochs', '3', '--learning_rate', '2e-5', '--warmup_ratio', '0.03', '--output_dir', '/mnt/bn/data-tns-live-llm/leon', 'datasets/p3_exp1', '--logging_steps', '100', '--save_strategy', 'steps', '--save_steps', '100', '--save_total_limit', '3', '--model_max_length', '1024']".replace("\'","\""))

["--model_name_or_path", "google/t5-xl-lm-adapt", "--data_path", "/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1.parquet", "--per_device_train_batch_size", "256", "--eval_strategy", "no", "--num_train_epochs", "3", "--learning_rate", "2e-5", "--warmup_ratio", "0.03", "--output_dir", "/mnt/bn/data-tns-live-llm/leon", "datasets/p3_exp1", "--logging_steps", "100", "--save_strategy", "steps", "--save_steps", "100", "--save_total_limit", "3", "--model_max_length", "1024"]


In [2]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [22]:
from transformers import AutoConfig, T5ForConditionalGeneration
import torch
import inspect
from accelerate import Accelerator
accelerator = Accelerator()
# config = AutoConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1")
model = T5ForConditionalGeneration.from_pretrained("google/t5-xl-lm-adapt", torch_dtype=torch.bfloat16)
accelerator.prepare(model)
accelerator.load_state("/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1/checkpoint-6/")
# signature = inspect.signature(model.forward)
# signature_columns = list(signature.parameters.keys())
# print(signature_columns)
print(model)

Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/bn/data-tns-live-llm/leon/datasets/p3_exp1/checkpoint-6/pytorch_model.bin'